In [222]:
# -*- coding: utf-8 -*-
import random
import torch
import math
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from sklearn import preprocessing
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support,roc_auc_score,balanced_accuracy_score,precision_recall_curve,auc,accuracy_score



In [226]:
import pickle
dataset = 'Afghanistan'
# dataset = 'EG'
# dataset = 'NI'
dataset = 'Syria'
dataset = 'Yemen'

with open('../data/{}/count_dataset.pkl'.format(dataset),'rb') as f:
    data = pickle.load(f)
# X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [227]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob=0.2):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
#         out = F.sigmoid(out)
        return out

 


In [228]:
#  torch.round(torch.tensor([0.51]))

In [229]:
[X_train, X_test, y_train, y_test] = data['temporal']



flat_X_train = X_train.reshape(X_train.shape[0],-1)
flat_X_test = X_test.reshape(X_test.shape[0],-1)

# print('flat_data_X',flat_data_X)
# print('X_train',X_train)
# flat_data_X2 = flat_data_X.reshape(X_train.shape)
# print('flat_data_X2',flat_data_X2)
scaler = preprocessing.StandardScaler().fit(flat_X_train)
X_scaled_train = scaler.transform(flat_X_train)
print(X_scaled_train.shape)
X_scaled_test = scaler.transform(flat_X_test)
X_train = X_scaled_train.reshape(X_train.shape)
X_test = X_scaled_test.reshape(X_test.shape)

X_train = torch.DoubleTensor(X_train)
X_test = torch.DoubleTensor(X_test)
y_test = torch.FloatTensor(y_test)
y_train = torch.FloatTensor(y_train) 
print(X_train.shape,'====')

(596, 350)
torch.Size([596, 14, 25]) ====


In [230]:

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

for x in batch(list(range(0, 10)), 3):
    print(x)
    

model = GRUModel(X_train.shape[-1], 32, 1,1 )
# X = torch.rand((120,14,1))
# Y = torch.rand((120,1))

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.BCEWithLogitsLoss() #reduction='sum'
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model.train()
for t in range(500):

    # Forward pass: Compute predicted y by passing x to the model
    loss_sum = 0
    for ids in batch(np.arange(len(X_train)), 16):
#         print(ids,X_train[ids].shape)
        inp = X_train[ids]
        out = y_train[ids].unsqueeze(-1)
        y_pred = model(inp.float())
    #     print(y_pred.shape,Y.shape)
        # Compute and print loss
        loss = criterion(y_pred, out)
        loss_sum += loss.item()
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if t %15 == 0:
        print(t, loss_sum)

# print(f'Result: {model.string()}')

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
[9]


/home/sdeng/anaconda2/envs/jupyter/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


0 23.817762106657028
15 12.695997681468725
30 3.9702672082930803
45 1.3020465387962759
60 0.15280283588799648
75 0.06640294900716981
90 0.03530067839892581
105 0.020867833158263238
120 0.013122031141392654
135 0.00864395374628657
150 0.0058777567764991545
165 0.004084490864897816
180 0.0028820754614571342
195 0.0020562701884045964
210 0.00147935111112929
225 0.0010709019177284063
240 0.0007789818544097216
255 0.0005689180636636593
270 0.0004166314976146168
285 0.00030578004509607126
300 0.00022488471842052604
315 0.00016548289903539626
330 0.00012189789600824952
345 8.993538608592644e-05
360 6.639926345997083e-05
375 4.894269457622613e-05
390 3.6194827615076974e-05
405 2.67028297713523e-05
420 1.967695558136029e-05
435 1.4580770905325835e-05
450 1.0699025422056252e-05
465 7.91996255422589e-06
480 5.8412526851014945e-06
495 4.291533045375218e-06


In [231]:
# rnn = nn.GRU(1, 16, 1,batch_first=True)
# input = torch.randn(5, 14, 1)
# # h0 = torch.randn(2, 3, 20)
# output, hn = rnn(input)
# (output.shape, hn.shape)
model.eval()
print(type(X_test))
X_test = X_test.float()
y_pred = model(X_test)

<class 'torch.Tensor'>


In [232]:
def evaluate(y_pred_tag, y_prob, y_test):
    bacc = balanced_accuracy_score(y_test, y_pred_tag)
    acc = accuracy_score(y_test, y_pred_tag)
    print('bacc',bacc,'acc',acc)
    precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
    # print(precision, recall, thresholds )
    area = auc(recall, precision)
    print("Area Under PR Curve(AP)", area)  #should be same as AP?
    aucv = roc_auc_score(y_test, y_prob)
    print('AUC',aucv)
#     print(type(y_test),type(y_pred))
    pre,rec, f1, _ = precision_recall_fscore_support(y_test, y_pred_tag, average='binary')
    print( 'pre {:.3f}  rec {:.3f}  f1 {:.3f}'.format(pre,rec, f1))
    return 

y_prob = torch.sigmoid(y_pred).detach().numpy()
y_pred_tag = np.round(y_prob) 
# y_prob = y_prob.numpy()
# y_test = y_test.numpy()

evaluate(y_pred_tag, y_prob , y_test )
# precision_recall_fscore_support(y_test, y_pred, average='binary')

bacc 0.5181603773584906 acc 0.6934673366834171
Area Under PR Curve(AP) 0.24130375017109607
AUC 0.6062893081761006
pre 0.231  rec 0.225  f1 0.228


In [233]:
[X_train, X_test, y_train, y_test] = data['static']



flat_X_train = X_train.reshape(X_train.shape[0],-1)
flat_X_test = X_test.reshape(X_test.shape[0],-1)
 
scaler = preprocessing.StandardScaler().fit(flat_X_train)
X_train = scaler.transform(flat_X_train)
X_test = scaler.transform(flat_X_test)

X_train = torch.DoubleTensor(X_train)
X_test = torch.DoubleTensor(X_test)
y_test = torch.FloatTensor(y_test)
y_train = torch.FloatTensor(y_train) 
print(X_train.shape,'====')

torch.Size([596, 350]) ====


In [234]:
clf = MLPClassifier(random_state=42, max_iter=500).fit(X_train, y_train)
# print(y_test.shape,y_prob.shape)
y_pred_tag = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1]

evaluate(y_pred_tag, y_prob , y_test ) 

bacc 0.6029088050314466 acc 0.7989949748743719
Area Under PR Curve(AP) 0.3517860592414753
AUC 0.6544025157232705
pre 0.500  rec 0.275  f1 0.355


In [235]:
clf = svm.SVC(probability=True)
clf.fit(X_train, y_train)
y_pred_tag = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1]

evaluate(y_pred_tag, y_prob , y_test ) 

bacc 0.5 acc 0.7989949748743719
Area Under PR Curve(AP) 0.3530952462511928
AUC 0.7250000000000001
pre 0.000  rec 0.000  f1 0.000


/home/sdeng/anaconda2/envs/jupyter/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [236]:
clf = LogisticRegression(random_state=42,max_iter=5000).fit(X_train, y_train)

y_pred_tag = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1]

evaluate(y_pred_tag, y_prob , y_test ) 

bacc 0.5869496855345913 acc 0.7286432160804021
Area Under PR Curve(AP) 0.3401168891621333
AUC 0.6042452830188679
pre 0.333  rec 0.350  f1 0.341


In [289]:
x = torch.randperm(100)[:3]
# x

params = {
        'balance1':[round(10**(i/2),6) for i in range(-10,7)],
        'balance2':[round(10**(i/2),6) for i in range(-10,7)],
        'rep_layer':[1,2,3],
        'hyp_layer':[1,2,3],
        'rep_dim':[20, 50, 100, 200],
        'hyp_dim':[20, 50, 100, 200],
        'batch':[100, 200]
    }
params.values()
keys = list(params.keys())
list(params.values())

[[1e-05,
  3.2e-05,
  0.0001,
  0.000316,
  0.001,
  0.003162,
  0.01,
  0.031623,
  0.1,
  0.316228,
  1.0,
  3.162278,
  10.0,
  31.622777,
  100.0,
  316.227766,
  1000.0],
 [1e-05,
  3.2e-05,
  0.0001,
  0.000316,
  0.001,
  0.003162,
  0.01,
  0.031623,
  0.1,
  0.316228,
  1.0,
  3.162278,
  10.0,
  31.622777,
  100.0,
  316.227766,
  1000.0],
 [1, 2, 3],
 [1, 2, 3],
 [20, 50, 100, 200],
 [20, 50, 100, 200],
 [100, 200]]

In [305]:
# for i in range(-10,7):
#     print(i/2,10**(i/2))
    
# xx = [round(10**(i/2),3) for i in range(-10,7)]
# xx
# itertools.combinations(iterable, r)
from itertools import product
 
ii=0
for combo in product(**params,repeat=1):
#     print(combo[0])
    arguments = {k: v[0] for k, v in zip(keys, combo) if v is not None}
    param_set = ''
    for k in arguments:
        param_set += ' --{} {}'.format(k,arguments[k])
    print(param_set)
    ii+=1
    

TypeError: product() takes at most 1 keyword argument (8 given)

In [298]:
d_rn = [64,32] # rnn hidden states
# d_pre = [64,32]
d_pre = [64]
d_c = [64,32]
d_r = [128,64]
d_h = [64,32]
d_g = [128,64]
keys = ['d_rn','d_pre','d_c','d_r','d_h','d_g']
for combo in product(d_rn, d_pre, d_c, d_r,d_h,d_g,repeat=1):
    arguments = {k: v for k, v in zip(keys, combo) if v is not None}
    param_set = ''
    for k in arguments:
        param_set += ' --{} {}'.format(k,arguments[k])
    print(param_set)


 --d_rn 64 --d_pre 64 --d_c 64 --d_r 128 --d_h 64 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 64 --d_r 128 --d_h 64 --d_g 64
 --d_rn 64 --d_pre 64 --d_c 64 --d_r 128 --d_h 32 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 64 --d_r 128 --d_h 32 --d_g 64
 --d_rn 64 --d_pre 64 --d_c 64 --d_r 64 --d_h 64 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 64 --d_r 64 --d_h 64 --d_g 64
 --d_rn 64 --d_pre 64 --d_c 64 --d_r 64 --d_h 32 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 64 --d_r 64 --d_h 32 --d_g 64
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 128 --d_h 64 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 128 --d_h 64 --d_g 64
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 128 --d_h 32 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 128 --d_h 32 --d_g 64
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 64 --d_h 64 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 64 --d_h 64 --d_g 64
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 64 --d_h 32 --d_g 128
 --d_rn 64 --d_pre 64 --d_c 32 --d_r 64 --d_h 32 --d_g 64
 --d_rn 32 --d_pre 64 --d_c 64 --d_r 128 --d_h 64 --d_g 

In [301]:
mapping = {'a': 5, 'c': 3, 'd': 9} 
bar(**mapping)

SyntaxError: invalid syntax (<ipython-input-301-ee433ea983c8>, line 2)